In [21]:
import os, datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow import keras
from utils.Dataset import CraneDataset
from utils.detect_roll_over import calculate_load_margin
from utils.assemble_model import build_model_v2
from tqdm.auto import tqdm
from utils.miscellaneous import create_seq_dataset_multiple_input_single_output

In [22]:
data_file_path_list = []
data_file_name_list = os.listdir('data')

for file_name in data_file_name_list:
    data_file_path_list.append(os.path.join('data', file_name))

In [23]:
dataset_inst = CraneDataset(data_file_path_list)
raw_dataset = dataset_inst.get_dataset()
dataset = raw_dataset.copy()
dataset = dataset.iloc[0::10]
dataset.reset_index(drop=True, inplace=True)

feature_name_list = dataset_inst.get_data_feature_names()
target_name_list = dataset_inst.get_data_target_names()

for target_name in target_name_list:
    dataset[target_name] = dataset[target_name]/1000

dataset = dataset[dataset['Time(sec)']>5]
dataset.reset_index(drop=True, inplace=True)

  0%|          | 0/22 [00:00<?, ?it/s]

In [24]:
dataset.describe()

,Time(sec),file_idx,safe_state,dataset_type,Boom_Angle(deg),Swing_Angle(deg),Load(Ton),Roll_Angle(deg),Yaw_Angle(deg),Pitch_Angle(deg),Actual_Load_Left_1(N),Actual_Load_Left_2(N),Actual_Load_Left_3(N),Actual_Load_Left_4(N),Actual_Load_Left_5(N),Actual_Load_Right_1(N),Actual_Load_Right_2(N),Actual_Load_Right_3(N),Actual_Load_Right_4(N),Actual_Load_Right_5(N)
count,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9.053000e+03,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000
mean,28.802418,8.470120,0.497294,0.204463,63.971458,7.868101e+01,76.677345,-0.298679,-0.006289,-0.094700,294.133486,286.106773,272.444717,257.072583,240.702073,216.120884,195.988137,178.972447,167.413640,164.019862
std,17.062724,6.446362,0.500020,0.403331,11.303686,6.254359e+01,23.485158,2.854945,0.562169,4.120054,130.810595,94.043262,74.707190,96.721019,133.998874,131.392459,92.842979,62.836182,86.308009,123.854206
min,5.000253,0.000000,0.000000,0.000000,40.000000,-4.900129e-24,40.000000,-55.226447,-12.851485,-60.990604,-0.193190,-0.161268,-0.133304,-0.154934,-0.176564,-0.189199,-0.194186,-0.199174,-0.204161,-0.209149
25%,15.290253,3.000000,0.000000,0.000000,60.000000,1.950487e+01,60.000000,-0.000205,-0.014990,-0.000288,195.890310,215.654440,220.467930,190.450210,125.067280,105.907280,136.703250,145.794730,113.637460,62.553569
50%,25.570253,6.000000,0.000000,0.000000,70.000000,7.229247e+01,70.000000,-0.000097,0.001809,-0.000053,323.060180,287.035430,262.174070,255.040010,232.190780,205.499800,194.406190,185.936720,153.338750,129.723400
75%,37.680253,14.000000,1.000000,0.000000,70.000000,1.350000e+02,90.000000,-0.000006,0.020865,0.000164,406.636090,351.652970,311.451150,322.151770,349.471120,320.441540,250.131390,210.716520,209.652670,247.547140
max,70.000000,21.000000,1.000000,1.000000,80.000000,1.800000e+02,120.000000,2.907156,8.315582,60.047735,500.241400,500.246570,500.271330,500.298090,500.262100,500.276140,500.286300,500.256560,500.207620,500.124620


In [25]:
left_load_name_list = ['Actual_Load_Left_1(N)', 'Actual_Load_Left_2(N)', 'Actual_Load_Left_3(N)', 'Actual_Load_Left_4(N)', 'Actual_Load_Left_5(N)']
right_load_name_list = ['Actual_Load_Right_1(N)', 'Actual_Load_Right_2(N)', 'Actual_Load_Right_3(N)', 'Actual_Load_Right_4(N)', 'Actual_Load_Right_5(N)']

front_load_name_list = ['Actual_Load_Left_1(N)', 'Actual_Load_Left_2(N)', 'Actual_Load_Right_1(N)', 'Actual_Load_Right_2(N)']
rear_load_name_list = ['Actual_Load_Left_4(N)', 'Actual_Load_Left_5(N)', 'Actual_Load_Right_4(N)', 'Actual_Load_Right_5(N)']

In [26]:
left_load_margin_list = []
right_load_margin_list = []
front_load_margin_list = []
rear_load_margin_list = []
roll_over_det_list = []

n_file = int(dataset['file_idx'].max()+1)

for file_idx in tqdm(range(n_file)):
    data_sample = dataset[dataset['file_idx']==file_idx]

    for i in range(data_sample.shape[0]):
        sample = data_sample.iloc[i, :]
        load_arr = sample[left_load_name_list+right_load_name_list].to_numpy()
        swing_angle = sample['Swing_Angle(deg)'].item()
        
        load_margin_array, rollover_det = calculate_load_margin(ground_load=load_arr, swing_angle=swing_angle)
        
        left_load_margin_list.append(load_margin_array[0])
        right_load_margin_list.append(load_margin_array[1])
        front_load_margin_list.append(load_margin_array[2])
        rear_load_margin_list.append(load_margin_array[3])
        roll_over_det_list.append(int(rollover_det))

left_load_margin_arr = np.array(left_load_margin_list)
right_load_margin_arr = np.array(right_load_margin_list)
front_load_margin_arr = np.array(front_load_margin_list)
rear_load_margin_arr = np.array(rear_load_margin_list)
roll_over_det_arr = np.array(roll_over_det_list)

dataset = pd.concat([dataset, pd.DataFrame(roll_over_det_arr, columns=['roll_over_det'])], axis=1)

  0%|          | 0/22 [00:00<?, ?it/s]

In [27]:
dataset.describe()

,Time(sec),file_idx,safe_state,dataset_type,Boom_Angle(deg),Swing_Angle(deg),Load(Ton),Roll_Angle(deg),Yaw_Angle(deg),Pitch_Angle(deg),...,Actual_Load_Left_2(N),Actual_Load_Left_3(N),Actual_Load_Left_4(N),Actual_Load_Left_5(N),Actual_Load_Right_1(N),Actual_Load_Right_2(N),Actual_Load_Right_3(N),Actual_Load_Right_4(N),Actual_Load_Right_5(N),roll_over_det
count,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9.053000e+03,9053.000000,9053.000000,9053.000000,9053.000000,...,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000,9053.000000
mean,28.802418,8.470120,0.497294,0.204463,63.971458,7.868101e+01,76.677345,-0.298679,-0.006289,-0.094700,...,286.106773,272.444717,257.072583,240.702073,216.120884,195.988137,178.972447,167.413640,164.019862,0.091351
std,17.062724,6.446362,0.500020,0.403331,11.303686,6.254359e+01,23.485158,2.854945,0.562169,4.120054,...,94.043262,74.707190,96.721019,133.998874,131.392459,92.842979,62.836182,86.308009,123.854206,0.288123
min,5.000253,0.000000,0.000000,0.000000,40.000000,-4.900129e-24,40.000000,-55.226447,-12.851485,-60.990604,...,-0.161268,-0.133304,-0.154934,-0.176564,-0.189199,-0.194186,-0.199174,-0.204161,-0.209149,0.000000
25%,15.290253,3.000000,0.000000,0.000000,60.000000,1.950487e+01,60.000000,-0.000205,-0.014990,-0.000288,...,215.654440,220.467930,190.450210,125.067280,105.907280,136.703250,145.794730,113.637460,62.553569,0.000000
50%,25.570253,6.000000,0.000000,0.000000,70.000000,7.229247e+01,70.000000,-0.000097,0.001809,-0.000053,...,287.035430,262.174070,255.040010,232.190780,205.499800,194.406190,185.936720,153.338750,129.723400,0.000000
75%,37.680253,14.000000,1.000000,0.000000,70.000000,1.350000e+02,90.000000,-0.000006,0.020865,0.000164,...,351.652970,311.451150,322.151770,349.471120,320.441540,250.131390,210.716520,209.652670,247.547140,0.000000
max,70.000000,21.000000,1.000000,1.000000,80.000000,1.800000e+02,120.000000,2.907156,8.315582,60.047735,...,500.246570,500.271330,500.298090,500.262100,500.276140,500.286300,500.256560,500.207620,500.124620,1.000000


In [28]:
train_feature = dataset[feature_name_list].to_numpy()
train_target = dataset['roll_over_det'].to_numpy()

n_samples = 1000
max_len = 1000

random_start_idx = np.random.randint(low=0, high=train_feature.shape[0]-max_len, size=n_samples)
random_len_idx = np.random.randint(low=300, high=max_len+1, size=n_samples)

feature_buf = np.empty(shape=(0, train_feature.shape[1]))
target_buf = np.empty(shape=(0,))

for i, j in zip(random_start_idx, random_len_idx):
    feature_buf = np.concatenate([feature_buf, train_feature[i:i+j]], axis=0)
    target_buf = np.concatenate([target_buf, train_target[i:i+j]], axis=0)

new_train_feature = feature_buf
new_train_target = target_buf

print(new_train_feature.shape, new_train_target.shape)

print((new_train_feature.itemsize*new_train_feature.size)/(1024**2))
print((new_train_target.itemsize*new_train_target.size)/(1024**2))

(646061, 6) (646061,)
29.574325561523438
4.929054260253906


In [29]:
new_train_target.max()

1.0

In [30]:
augment_dataset = np.concatenate([new_train_feature, new_train_target.reshape(-1, 1)], axis=1)

seq_len = 30
pred_distance = 30

train_feature, train_target = create_seq_dataset_multiple_input_single_output(data=augment_dataset,
                                                                              seq_len=seq_len,
                                                                              pred_distance=pred_distance,
                                                                              target_idx_pos=augment_dataset.shape[1]-1)
train_target = np.squeeze(train_target)
print(train_feature.shape, train_target.shape)

print((train_feature.itemsize*train_feature.size)/(1024**2))
print((train_target.itemsize*train_target.size)/(1024**2))

creating sequence dataset...:   0%|          | 0/646031 [00:00<?, ?it/s]

(646002, 30, 6) (646002,)
887.1487426757812
4.9286041259765625


In [31]:
model = build_model_v2(input_shape=train_feature.shape[1:],d_dims=64, dropout_rate=0.4, learning_rate=0.001)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 6)]      0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 30, 6)       12          ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 dense (Dense)                  (None, 30, 64)       448         ['layer_normalization[0][0]']    
                                                                                                  
 batch_normalization (BatchNorm  (None, 30, 64)      256         ['dense[0][0]']              

In [ ]:
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=500, verbose=0)
csv_logger = keras.callbacks.CSVLogger('result/log.csv', append=False, separator=',')
model_chk_point = keras.callbacks.ModelCheckpoint(filepath='models/model.keras', monitor="val_loss", verbose=2, save_best_only=True,
                                                  save_weights_only=False, mode="min", save_freq="epoch", initial_value_threshold=None)

model.fit(x=train_feature, y=train_target, validation_data=(train_feature, train_target), epochs=999999,
          batch_size=5000, verbose=0, callbacks=[early_stop, model_chk_point, tensorboard_callback])


Epoch 1: val_loss improved from inf to 0.34559, saving model to models\model.keras


In [ ]:
#model.save('models/model_last.keras')

In [ ]:
 # plt.figure()
    #
    # for i in range(len(left_load_name_list)):
    #     sns.lineplot(x=time_arr, y=data_sample[left_load_name_list[i]], label=f'left Load {i+1}')
    #
    # sns.lineplot(x=time_arr, y=roll_over_det_list*500, label='Detection')
    #
    # plt.xlabel('Time (sec)')
    # plt.ylim(0, 500)
    # plt.ylabel('Ground Load (kN)')
    #
    # plt.savefig(os.path.join('results', f'left_load_{file_idx}.jpg'), dpi=600, bbox_inches='tight')
    #
    # plt.figure()
    #
    # for i in range(len(right_load_name_list)):
    #     sns.lineplot(x=time_arr, y=data_sample[right_load_name_list[i]], label=f'right Load {i+1}')
    #
    # sns.lineplot(x=time_arr, y=roll_over_det_list*500, label='Detection')
    #
    # plt.xlabel('Time (sec)')
    # plt.ylim(0, 500)
    # plt.ylabel('Ground Load (kN)')
    #
    # plt.savefig(os.path.join('results', f'right_load_{file_idx}.jpg'), dpi=600, bbox_inches='tight')

In [ ]:
for i in range(len(left_load_name_list)):
    sns.lineplot(x=time_arr, y=data_sample[left_load_name_list[i]], label=f'left Load {i+1}')

sns.lineplot(x=time_arr, y=roll_over_det_list*500, label='Detection')

plt.xlabel('Time (sec)')
plt.ylabel('Ground Load (kN)')

In [ ]:
for i in range(len(right_load_name_list)):
    sns.lineplot(x=time_arr, y=data_sample[right_load_name_list[i]], label=f'right Load {i+1}')

sns.lineplot(x=time_arr, y=roll_over_det_list*500, label='Detection')

plt.xlabel('Time (sec)')
plt.ylabel('Ground Load (kN)')

In [ ]:
plt.savefig("bar_plot.png", dpi=300, bbox_inches='tight')  # PNG 파일로 저장

In [ ]:
sns.set_theme(font_scale=1.0)
sns.set_style("whitegrid", {'grid.linestyle': '--'})

sns.lineplot(x=time_arr, y=left_load_margin_list, label='Left Load Margin')


In [ ]:
sns.set_theme(font_scale=1.0)
sns.set_style("whitegrid", {'grid.linestyle': '--'})

sns.lineplot(x=time_arr, y=right_load_margin_list, label='Right Load Margin')


In [ ]:
sns.set_theme(font_scale=1.0)
sns.set_style("whitegrid", {'grid.linestyle': '--'})

sns.lineplot(x=time_arr, y=rear_load_margin_list, label='Rear Load Margin')

In [ ]:
sns.set_theme(font_scale=1.0)
sns.set_style("whitegrid", {'grid.linestyle': '--'})

sns.lineplot(x=time_arr, y=rear_load_margin_list, label='Rear Load Margin')